# Simple Mixed Integer Programming (MIP) Example with PuLP

cuOpt is NVIDIA's GPU accelerated solver that delivers massive speedups for real-world LP, MIP, and VRP workloads.

cuOpt seemlessly integrates with modeling languages. You can drop cuOpt into existing models built with PuLP and AMPL, with minimal refactoring. Let's take a look at an example solving a simple MIP problem with cuOpt.

To run this in Google Colab, download the notebook and upload it to Google Colab. Make sure you are running this on a T4 GPU.

If you are running this in the cuOpt container, you are good to go!


## 1. Install Dependencies

To make sure we are good to go, let's install PuLP and cuOpt.

__[PuLP](https://coin-or.github.io/pulp/)__ is a popular linear and mixed integer programming modeler written in Python.


If you are running this notebook in Google Colab, or elsewhere outside the container where cuOpt is not yet installed, uncomment the pip install command to install cuOpt.

In [ ]:
import subprocess
from IPython.display import display, HTML

def check_gpu():
    try:
        output = subprocess.check_output(["nvidia-smi"], shell=False, stderr=subprocess.STDOUT).decode()
        lines = output.splitlines()
        gpu_info = lines[2] if len(lines) > 2 else "GPU detected"
        display(HTML(f"""
        <div style="border:2px solid #4CAF50;padding:10px;border-radius:10px;background:#e8f5e9;">
            <h3>✅ GPU is enabled</h3>
            <pre>{gpu_info}</pre>
        </div>
        """))
    except (subprocess.CalledProcessError, FileNotFoundError, IndexError) as e:
        display(HTML("""
        <div style="border:2px solid red;padding:15px;border-radius:10px;background:#ffeeee;">
            <h3>⚠️ GPU not detected!</h3>
            <p>This notebook requires a <b>GPU runtime</b>.</p>
            
            <h4>If running in Google Colab:</h4>
            <ol>
              <li>Click on <b>Runtime → Change runtime type</b></li>
              <li>Set <b>Hardware accelerator</b> to <b>GPU</b></li>
              <li>Then click <b>Save</b> and <b>Runtime → Restart runtime</b>.</li>
            </ol>
            
            <h4>If running in Docker:</h4>
            <ol>
              <li>Ensure you have <b>NVIDIA Docker runtime</b> installed (<code>nvidia-docker2</code>)</li>
              <li>Run container with GPU support: <code>docker run --gpus all ...</code></li>
              <li>Or use: <code>docker run --runtime=nvidia ...</code> for older Docker versions</li>
              <li>Verify GPU access: <code>docker run --gpus all nvidia/cuda:12.0.0-base-ubuntu22.04 nvidia-smi</code></li>
            </ol>
            
            <p><b>Additional resources:</b></p>
            <ul>
              <li><a href="https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/install-guide.html" target="_blank">NVIDIA Container Toolkit Installation Guide</a></li>
            </ul>
        </div>
        """))

check_gpu()

In [ ]:
pip install pulp==3.2.0

In [ ]:
# # Enable this in case you are running this in google colab or such places where cuOpt is not yet installed

#!pip install --upgrade --extra-index-url=https://pypi.nvidia.com cuopt-cu12 nvidia-nvjitlink-cu12 rapids-logger==0.1.19
#!pip install --upgrade --extra-index-url=https://pypi.nvidia.com cuopt-cu13 nvidia-nvjitlink-cu13 rapids-logger==0.1.19

## 2. Problem Setup

In this example, the goal is to minimize the objective function 2x+3y, where x is an integer variable and y is a continuous variable constrained to be non-negative. The problem is subject to two constraints: x+y≥10 and x≤15.

In [ ]:
from pulp import *

# Define the problem
problem = LpProblem("Integer_Optimization", LpMinimize)

# Define variables
x = LpVariable('x', cat='Integer')         # Integer
y = LpVariable('y', lowBound=0.0)            # Non-negative

# Objective function
problem += 2.0 * x + 3.0 * y, "Objective"

# Constraints
problem += x + y >= 10.0, "Constraint1"
problem += x <= 15.0, "Constraint2"


## 3. Problem Solution

PuLP calls on the cuOpt solver, which finds the optimal values of x and y that minimize the objective while satisfying the constraints.

In [ ]:

# Solve the problem using CUOPT
status = problem.solve(CUOPT(msg=0))

# Print results
print("Status:", LpStatus[status])
print("Optimal Value:", value(problem.objective))
print("x =", x.varValue)
print("y =", y.varValue)

We can see that cuOpt quickly solves the problem, with the final solution being x = 10.0
y = 0.0